In [7]:
# Import dependencies
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [10]:
# Read in data for Per capita income by city
All_Homes_df = pd.read_csv("../Resources/City_Zhvi_AllHomes.csv")
All_Homes_df

,RegionID,SizeRank,RegionName,RegionType,State,StateName,CountyName,Jan-10,Feb-10,Mar-10,...,Jun-19,Jul-19,Aug-19,Sep-19,Oct-19,Nov-19,Dec-19,Jan-20,Feb-20,Mar-20
0,6181,0,New York,City,NY,New York,Queens County,444369.0,444560.0,443163.0,...,659421,659007,658239,656925,655613,654394,653930,653901,653565,652307
1,12447,1,Los Angeles,City,CA,California,Los Angeles County,443474.0,441260.0,440817.0,...,712660,713807,715688,718245,721896,725180,730358,735910,744137,752508
2,39051,2,Houston,City,TX,Texas,Harris County,127564.0,127372.0,127497.0,...,186844,187464,188070,188496,189125,189612,190179,190395,190938,191907
3,17426,3,Chicago,City,IL,Illinois,Cook County,217978.0,218203.0,215311.0,...,248372,248646,248725,248483,248278,248090,248029,248220,248599,249152
4,6915,4,San Antonio,City,TX,Texas,Bexar County,127517.0,127650.0,127677.0,...,182732,183350,183930,184846,185490,186244,186420,186962,187129,187718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27325,48550,29449,Winton,City,MN,Minnesota,Saint Louis County,78486.0,79689.0,80756.0,...,70929,70176,68979,68537,68948,69481,69423,69355,69654,70399
27326,51821,29449,Eastabuchie,City,MS,Mississippi,Jones County,66148.0,65857.0,65907.0,...,77737,78461,79138,79930,79683,79838,80272,80969,81871,82284
27327,232607,29449,Dean,City,TX,Texas,Clay County,124299.0,124047.0,123632.0,...,178872,179976,180654,181188,181613,182101,182424,182764,182639,182361
27328,6638,29449,Pulaski,City,GA,Georgia,Candler County,70380.0,70546.0,70657.0,...,97356,97757,98014,98225,98303,98613,98802,99227,99699,100464


In [12]:
All_Homes_df.drop_duplicates(subset="RegionName",keep = "first", inplace = True)
len(All_Homes_df)


17616

In [25]:
All_Homes_df.drop('RegionID', axis=1, inplace=True)
All_Homes_df

,SizeRank,RegionName,RegionType,State,StateName,CountyName,Jan-10,Feb-10,Mar-10,Apr-10,...,Jun-19,Jul-19,Aug-19,Sep-19,Oct-19,Nov-19,Dec-19,Jan-20,Feb-20,Mar-20
0,0,New York,City,NY,New York,Queens County,444369.0,444560.0,443163.0,442489.0,...,659421,659007,658239,656925,655613,654394,653930,653901,653565,652307
1,1,Los Angeles,City,CA,California,Los Angeles County,443474.0,441260.0,440817.0,441873.0,...,712660,713807,715688,718245,721896,725180,730358,735910,744137,752508
2,2,Houston,City,TX,Texas,Harris County,127564.0,127372.0,127497.0,127711.0,...,186844,187464,188070,188496,189125,189612,190179,190395,190938,191907
3,3,Chicago,City,IL,Illinois,Cook County,217978.0,218203.0,215311.0,215345.0,...,248372,248646,248725,248483,248278,248090,248029,248220,248599,249152
4,4,San Antonio,City,TX,Texas,Bexar County,127517.0,127650.0,127677.0,127675.0,...,182732,183350,183930,184846,185490,186244,186420,186962,187129,187718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27322,29449,Columbus Township,City,PA,Pennsylvania,Warren County,NaN,NaN,NaN,NaN,...,85912,86201,86514,87365,87234,87160,86902,87095,87705,87924
27323,29449,Pursglove,City,WV,West Virginia,Monongalia County,NaN,NaN,NaN,NaN,...,35891,36054,36024,36034,35844,36001,36098,36205,36059,35973
27324,29449,Tome,City,NM,New Mexico,Valencia County,NaN,NaN,NaN,NaN,...,124700,124903,125228,125531,125908,126784,127890,129247,129959,131079
27326,29449,Eastabuchie,City,MS,Mississippi,Jones County,66148.0,65857.0,65907.0,66002.0,...,77737,78461,79138,79930,79683,79838,80272,80969,81871,82284


In [27]:
All_Homes_df.drop('State', axis=1, inplace=True)
All_Homes_df

,SizeRank,RegionName,RegionType,StateName,CountyName,Jan-10,Feb-10,Mar-10,Apr-10,May-10,...,Jun-19,Jul-19,Aug-19,Sep-19,Oct-19,Nov-19,Dec-19,Jan-20,Feb-20,Mar-20
0,0,New York,City,New York,Queens County,444369.0,444560.0,443163.0,442489.0,442178.0,...,659421,659007,658239,656925,655613,654394,653930,653901,653565,652307
1,1,Los Angeles,City,California,Los Angeles County,443474.0,441260.0,440817.0,441873.0,445830.0,...,712660,713807,715688,718245,721896,725180,730358,735910,744137,752508
2,2,Houston,City,Texas,Harris County,127564.0,127372.0,127497.0,127711.0,127788.0,...,186844,187464,188070,188496,189125,189612,190179,190395,190938,191907
3,3,Chicago,City,Illinois,Cook County,217978.0,218203.0,215311.0,215345.0,212877.0,...,248372,248646,248725,248483,248278,248090,248029,248220,248599,249152
4,4,San Antonio,City,Texas,Bexar County,127517.0,127650.0,127677.0,127675.0,127681.0,...,182732,183350,183930,184846,185490,186244,186420,186962,187129,187718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27322,29449,Columbus Township,City,Pennsylvania,Warren County,NaN,NaN,NaN,NaN,NaN,...,85912,86201,86514,87365,87234,87160,86902,87095,87705,87924
27323,29449,Pursglove,City,West Virginia,Monongalia County,NaN,NaN,NaN,NaN,NaN,...,35891,36054,36024,36034,35844,36001,36098,36205,36059,35973
27324,29449,Tome,City,New Mexico,Valencia County,NaN,NaN,NaN,NaN,NaN,...,124700,124903,125228,125531,125908,126784,127890,129247,129959,131079
27326,29449,Eastabuchie,City,Mississippi,Jones County,66148.0,65857.0,65907.0,66002.0,66188.0,...,77737,78461,79138,79930,79683,79838,80272,80969,81871,82284


In [28]:
All_Homes_df.drop('RegionType', axis=1, inplace=True)
All_Homes_df

,SizeRank,RegionName,StateName,CountyName,Jan-10,Feb-10,Mar-10,Apr-10,May-10,Jun-10,...,Jun-19,Jul-19,Aug-19,Sep-19,Oct-19,Nov-19,Dec-19,Jan-20,Feb-20,Mar-20
0,0,New York,New York,Queens County,444369.0,444560.0,443163.0,442489.0,442178.0,444115.0,...,659421,659007,658239,656925,655613,654394,653930,653901,653565,652307
1,1,Los Angeles,California,Los Angeles County,443474.0,441260.0,440817.0,441873.0,445830.0,444219.0,...,712660,713807,715688,718245,721896,725180,730358,735910,744137,752508
2,2,Houston,Texas,Harris County,127564.0,127372.0,127497.0,127711.0,127788.0,127292.0,...,186844,187464,188070,188496,189125,189612,190179,190395,190938,191907
3,3,Chicago,Illinois,Cook County,217978.0,218203.0,215311.0,215345.0,212877.0,213038.0,...,248372,248646,248725,248483,248278,248090,248029,248220,248599,249152
4,4,San Antonio,Texas,Bexar County,127517.0,127650.0,127677.0,127675.0,127681.0,127691.0,...,182732,183350,183930,184846,185490,186244,186420,186962,187129,187718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27322,29449,Columbus Township,Pennsylvania,Warren County,NaN,NaN,NaN,NaN,NaN,NaN,...,85912,86201,86514,87365,87234,87160,86902,87095,87705,87924
27323,29449,Pursglove,West Virginia,Monongalia County,NaN,NaN,NaN,NaN,NaN,NaN,...,35891,36054,36024,36034,35844,36001,36098,36205,36059,35973
27324,29449,Tome,New Mexico,Valencia County,NaN,NaN,NaN,NaN,NaN,NaN,...,124700,124903,125228,125531,125908,126784,127890,129247,129959,131079
27326,29449,Eastabuchie,Mississippi,Jones County,66148.0,65857.0,65907.0,66002.0,66188.0,66299.0,...,77737,78461,79138,79930,79683,79838,80272,80969,81871,82284


In [29]:
#Check for nulls
for column in All_Homes_df.columns:
    print(f"column {column} has {All_Homes_df[column].isnull().sum()} null values")

column SizeRank has 0 null values
column RegionName has 0 null values
column StateName has 0 null values
column CountyName has 0 null values
column Jan-10 has 2169 null values
column Feb-10 has 2141 null values
column Mar-10 has 2131 null values
column Apr-10 has 2111 null values
column May-10 has 2090 null values
column Jun-10 has 2070 null values
column Jul-10 has 2060 null values
column Aug-10 has 2032 null values
column Sep-10 has 2011 null values
column Oct-10 has 2000 null values
column Nov-10 has 1985 null values
column Dec-10 has 1979 null values
column Jan-11 has 1912 null values
column Feb-11 has 1894 null values
column Mar-11 has 1862 null values
column Apr-11 has 1834 null values
column May-11 has 1818 null values
column Jun-11 has 1811 null values
column Jul-11 has 1796 null values
column Aug-11 has 1777 null values
column Sep-11 has 1758 null values
column Oct-11 has 1740 null values
column Nov-11 has 1730 null values
column Dec-11 has 1718 null values
column Jan-12 has 1

In [31]:
All_Homes_df["StateName"].unique()

array(['New York', 'California', 'Texas', 'Illinois', 'Pennsylvania',
       'Arizona', 'Nevada', 'Florida', 'Indiana', 'North Carolina',
       'Ohio', 'Kentucky', 'Michigan', 'Colorado', 'Washington',
       'Tennessee', 'Massachusetts', 'District of Columbia', 'Oregon',
       'Maryland', 'Wisconsin', 'Nebraska', 'New Mexico', 'Oklahoma',
       'Georgia', 'Missouri', 'Virginia', 'Minnesota', 'Hawaii',
       'Louisiana', 'Kansas', 'South Carolina', 'Alaska', 'New Jersey',
       'Alabama', 'Idaho', 'Iowa', 'Utah', 'Arkansas', 'Rhode Island',
       'South Dakota', 'Mississippi', 'Connecticut', 'Montana',
       'New Hampshire', 'North Dakota', 'Wyoming', 'Delaware',
       'West Virginia', 'Maine', 'Vermont'], dtype=object)

In [32]:
All_Homes_df["StateName"].value_counts()

Pennsylvania            1392
Texas                   1104
New York                 959
California               924
Illinois                 727
Ohio                     707
Florida                  569
Missouri                 567
North Carolina           555
Michigan                 554
Minnesota                545
Wisconsin                506
Virginia                 471
Iowa                     455
New Jersey               447
Georgia                  411
Oklahoma                 406
Kentucky                 395
Alabama                  358
Arkansas                 353
Indiana                  352
Washington               333
West Virginia            320
Maryland                 311
Tennessee                310
Louisiana                274
Colorado                 261
Oregon                   236
South Carolina           235
Massachusetts            235
Kansas                   218
Maine                    207
Mississippi              201
Nebraska                 169
Idaho         

In [37]:
All_Homes_cleansed_df = All_Homes_df.dropna(how='any',axis=0) 
All_Homes_cleansed_df

,SizeRank,RegionName,StateName,CountyName,Jan-10,Feb-10,Mar-10,Apr-10,May-10,Jun-10,...,Jun-19,Jul-19,Aug-19,Sep-19,Oct-19,Nov-19,Dec-19,Jan-20,Feb-20,Mar-20
0,0,New York,New York,Queens County,444369.0,444560.0,443163.0,442489.0,442178.0,444115.0,...,659421,659007,658239,656925,655613,654394,653930,653901,653565,652307
1,1,Los Angeles,California,Los Angeles County,443474.0,441260.0,440817.0,441873.0,445830.0,444219.0,...,712660,713807,715688,718245,721896,725180,730358,735910,744137,752508
2,2,Houston,Texas,Harris County,127564.0,127372.0,127497.0,127711.0,127788.0,127292.0,...,186844,187464,188070,188496,189125,189612,190179,190395,190938,191907
3,3,Chicago,Illinois,Cook County,217978.0,218203.0,215311.0,215345.0,212877.0,213038.0,...,248372,248646,248725,248483,248278,248090,248029,248220,248599,249152
4,4,San Antonio,Texas,Bexar County,127517.0,127650.0,127677.0,127675.0,127681.0,127691.0,...,182732,183350,183930,184846,185490,186244,186420,186962,187129,187718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27315,29449,Village Mills,Texas,Tyler County,117190.0,117071.0,116919.0,117195.0,117212.0,117384.0,...,172467,171994,171426,171730,172091,172584,172546,173565,173906,174558
27316,29449,Derby Center,Vermont,Orleans County,150185.0,150015.0,149546.0,149496.0,149722.0,149751.0,...,169819,170361,170714,171388,171764,172331,172573,173269,173510,173921
27319,29449,Port Mansfield,Texas,Willacy County,107805.0,106936.0,106011.0,105720.0,105475.0,105189.0,...,153588,154334,154956,156258,157394,158677,159269,159540,159186,159223
27326,29449,Eastabuchie,Mississippi,Jones County,66148.0,65857.0,65907.0,66002.0,66188.0,66299.0,...,77737,78461,79138,79930,79683,79838,80272,80969,81871,82284
